Copyright (c) 2020 Martin Holle. Alle Rechte vorbehalten. Lizensiert unter der MIT-Lizenz.

# Covid-19 Statistics Aachen: Datenabfrage

Abfrage der Daten von der Website der Städteregion Aachen und Speichern in einer Excel-Datei für die Datenübergabe an den nächsten Schritt, in dem die Daten aufbereitet werden.

## Vorbereitungen

- Benötigte Imports
- Konfiguration aus zentraler `.ini`-Datei einlesen
- Konfiguration und Instanzierung des Loggers
- Globale Variablen definieren

In [2]:
import pandas as pd
import numpy as np

import re
from datetime import date
from datetime import timedelta
from datetime import date
from datetime import time
from datetime import datetime

import requests
from bs4 import BeautifulSoup

import logging
import configparser

# Konfiguration einlesen
config = configparser.ConfigParser()
config.read('config.ini')

# Konfiguration des Loggings
# - Die Logging-Ausgaben werden in der lokalen Datei covid-19-datenabfrage.log geschrieben
# - Für die Ausgabe wird eine bestimmte Formatierung konfiguriert
fhandler = logging.FileHandler(filename=config['Logging']['LogFileName'], mode='a')

# TODO: Formatierung finalisieren (Tausendstel-Sekunden, Tag des Monats, 1. Zeichen des Levels)
formatter = logging.Formatter('%(asctime)s %(levelname)-1.1s %(name)-20.20s - %(message)s')
fhandler.setFormatter(formatter)

# Instanzierung und Konfigurierung des Loggers
log = logging.getLogger("datenabfrage")
log.addHandler(fhandler)
log.setLevel(logging.DEBUG)

# Für die Zwischenspeicherung des eingelesenen HTML
# - Wenn die Website in der aktuellen Session schon einmal abgefragt wurde, wird das Ergebnis
#   der Abfrage in dieser Variablen gesichert
# - Dies erleichtert die Entwicklung der nachfolgenden Verabeitungsschritte und führt nicht immer wieder zu
#   neuen überflüssigen Abfragen der Website
html_payload = None

## Einlesen der existierenden Excel-Datei

- Datei und Seite der Excel-Datei: Siehe `config.ini`
- Einzulesende Spalten: 
  - **A**: Datum im Format 'DD.MM.'
  - **B**: Akkumulierte Anzahl der Infektionen für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **C**: Akkumulierte Anzahl der Infektionen für die Stadt Aachen als Integerzahl
  - **D**: Anzahl neuer Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **E**: Akkumulierte Anzahl der Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl 
  - **F**: Akkumulierte Anzahl der Genesenen für gesamte Städteregion (inkl. Aachen) als Integerzahl
- Spalte A als Datum interpretieren
- Die erste Zeile (Header) überspringen
- Label der Spalten explizit setzen

In [3]:
col_names = ['Uhrzeit', 'Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ]

try:
    c19_cases = pd.read_excel(config['Rohdaten']['FileName'], 
                              sheet_name=config['Rohdaten']['SheetName'], 
                              index_col=0,
                              parse_dates=[0],
                              skiprows=[],
                              names=col_names)
except FileNotFoundError as err: 
    log.warning('Error during pd.read_excel(): {0}'.format(err))
    # Leere DataFrame für den Start erzeugen
    c19_cases = pd.DataFrame(columns=col_names, index=pd.DatetimeIndex([], name='Datum'))
    
c19_cases.tail(14)

,Uhrzeit,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle
Datum,,,,,,
2020-10-06,09:00:00,2891,1445,107,2601,183
2020-10-07,11:45:00,2936,1468,107,2617,212
2020-10-08,10:30:00,2988,1496,107,2638,243
2020-10-09,10:00:00,3027,1515,109,2664,254
2020-10-12,11:15:00,3167,1585,109,2723,335
2020-10-13,11:45:00,3188,1594,109,2772,307
2020-10-14,11:45:00,3257,1626,109,2806,342
2020-10-15,10:15:00,3393,1672,109,2845,439
2020-10-16,12:15:00,3509,1729,109,2891,509


## Datenabfrage

### Funktionen für die Datenabfrage und Extraktion der Rohtexte

- `robot_access_allowed()` - Via robots.txt prüfen, ob die Website durch ein Skript abgefragt und verabeitet werden darf
- `gather_html()` - Website abfragen und HTML zurückliefern
- `gather_text()` - Relevante Texte aus dem von der Website geliefertem HTML extrahieren und zurückliefern

*<u>Anmerkungen:</u>  
Die Website der Städteregion Aachen verwendet keine `robots.txt`, damit wäre die Funktion `robot_access_allowed()` eigentlich überflüssig. Hier ist sie nur der Vollständigkeit halber definiert und nicht ausimplementiert.*

*Eine leicht andere Struktur bei der Meldung am 12.08.2020 erforderte auch eine Änderung an den CSS-Selektor in `gather_html()`.*

In [4]:
default_user_agent = config['Rohdaten']['UserAgent']

def robot_access_allowed(url: str, user_agent: str=default_user_agent) -> bool:
    """
    Return True if scraping is allowed according to robots.txt.
    
    Request robots.txt. If exist, parse robots.txt and return True if scraping by this script is allowed.
    
    Parameters
    ----------
    url: str
        URL of the website, which is to be scraped
    user_agent: str
        User agent string, which will be used when requesting the website
    
    Returns
    -------
    bool
        True, if web scraping is allowed by robots.txt or if robots.txt does not exist.
        False, if web scraping is not allowed by robots.txt
    """
    
    log.debug("access_allowed(" + url + ", " + user_agent + ")")
    
    headers = { 'user-agent': user_agent }
    
    # Extract root of the website's URL
    
    # Request robots.txt from root of the website
    
    # If robots.txt exist, parse content of the file
    
    return True

def gather_html(url: str, user_agent: str=default_user_agent) -> str:
    """
    Request website from <url> and return the HTML delivered by the website as text.

    Parameters
    ----------
    url: str
        URL of the website, which is to be scraped
    user_agent: str
        User agent string, which will be used when requesting the website
    
    Returns
    -------
    str
        The HTML which was gathered from the website
    """

    log.debug("gather_html(" + url + ", " + user_agent + ")")
    
    # Website abfragen
    headers = { 'user-agent': user_agent }
    page = requests.get(url)
    log.debug("gather_html/page.status_code: %d", page.status_code)
    for key, val in page.headers.items():
        log.debug("gather_html/page.header(%s): %s", key, val)   
    if page.status_code == requests.codes.ALL_OK: # Alles ok
        return page.text
        
    return None

def gather_text(html_text: str) -> []:
    """
    Extract relevant text content from HTML and return extracted text for news entry parts.
    
    Extract relevant text content, i.e. the distinctive news entries, from <html_text>. 
    Return array with news entries, each entry consisting of a dictionary with extracted text parts 
    for 'Header', 'Abstract', 'Main'.

    Parameters
    ----------
    html_str: str
        The HTML code, which was gathered from the website.
        
    Returns
    -------
    []
        Array with entries, each consisting of a dictionary with extracted text for 'Header', 'Abstract', 'Main' 
        part of the news entries.
    """

    log.debug(f"gather_text(%d chars)", len(html_text))

    records = []
       
    # Parser instanzieren
    soup = BeautifulSoup(html_text, 'html.parser')

    # Relevante Objekte aus geliefertem HTML extrahieren
    # - Header
    # - Abstract (existiert nur für die aktuellen Meldungen, nicht im Meldungsarchiv)
    # - Main
    divs = soup.select('div.mid-col article > div.textcontent > div')
    for div in divs:
        # Header erkennen: Nun wenn dieser gefunden wird, macht es Sinn, nachfolgend nach 
        # Abstract und Haupttext zu suchen
        header = div.select('h2')
        if header:
            # Header: Text extrahieren
            header_text = next(header[0].stripped_strings)

            # TODO: Ab hier überarbeiten, sodass der gesamte Body-Text eines Eintrags eingelesen wird
            
            body_text = ''
            body_elems = div.select('div > div.ce-bodytext > p, div > div.ce-bodytext > ul')
            for be in body_elems:
                # Body: Text extrahieren
                body_text += next(be.stripped_strings)
                print(body_text)

            # Neuen Eintrag mit den extahierten Texten hinzufügen 
            records.append({'Header': header_text, 'Body': body_text })
                        
    log.debug("gather_text: %d records extracted from HTML", len(records))
    
    return records

### Funktionen für die Extraktion der Falldaten aus den Rohtexten

In der ersten Version erfolgt die Extraktion der Daten aus den Texten mit Hilfe regulärer Ausdrücke:

- Datum/Uhrzeit der Meldung wird aus dem `Header` extrahiert.
- Die eigentlichen Daten werden zweimal extrahiert: Einmal aus dem `Abstract` und zum zweiten Mal aus dem `Main`-Text. 
- Falls sowohl `Abstract` als auch `Main` eingelesen wurden, werden anschließend die extrahierten Daten miteinander verglichen. Nur wenn sie übereinstimmen, wird der Datensatz übernommen.

Die für das Parsen der eingelesenen Texte verwendeten regulären Ausdrücke versuchen einerseits, auf Nummer sicher zu gehen, um die richtigen Textstellen für das Einlesen der Zahlen zu treffen, und andererseits einige Freiheitsgrade zuzulassen: 

- Leerzeichen und andere "White Spaces" werden von Menschen hin und wieder vergesssen oder mehrfach eingegeben
- Textvariationen kommen vor, in denen einzelne Wörter nicht erscheinen oder hinzugefügt werden
- Manchmal werden einzelne Buchstaben weggelassen, ohne dass dies ein Schreibfehler wäre
- Und natürlich kommen auch Schreibfehler vor 

Die regulären Ausdrücke sind dadurch relativ unübersichtlich geworden und berücksichtigen dennoch nicht alle Situationen. Beispielsweise wurde bis zum 10.08.2020 im `Abstract` die folgende Formulierung verwendet:

> Aktuell 2157 bestätigte Coronafälle in der StädteRegion Aachen (davon 1060 in der Stadt Aachen).

Der `Main`-Text sah ähnlich, aber ein wenig anders aus:

> Es gibt insgesamt in der StädteRegion nunmehr 2157 positive Fälle, davon 1060 in der Stadt Aachen.

Am 10.08.2020 wurde dann die Formulierung für den `Main`-Text minimal variert (Klammerausdruck statt Nebensatz):

> Es gibt insgesamt in der StädteRegion nunmehr 2170 positive Fälle (davon 1068 in der Stadt Aachen). 

Damit passte der reguläre Ausdruck nicht mehr, den ich dann anpassen musste. Obwohl reguläre Ausdrücke ein sehr mächtiges Mittel darstellen, um Texte zu verarbeiten, bleiben sie zwangsläufig auf der Ebene eines reinen Mustervergleichs stehen, die fehlerresistente Verarbeitung natürlicher Sprache stößt mit regulären Ausdrücken immer wieder an eine Grenze.

Ab dem 12.08.2020 wurden die Zahlen für die Stadt Aachen nicht mehr wie oben angegeben. Stattdessen, und nur im `Main`-Text der Meldung, wird nun eine Liste aller Kommunen in der Städteregion mit den jeweils auf sie entfallenden Fallzahlen mitgeteilt. Die Extraktion und Verarbeitung der Zahlen für die Stadt Aachen wurde daher zunächst deaktiviert und dann durch ein neues Verfahren ersetzt. Ebenfalls wurden das Verfahren und die regulären Ausdrücke für die Ermittlung der Todesfälle angepasst.

In [6]:
class NewsMeta():
    """Value class for meta data of news entry."""
    pass

class CaseFigures():
    """Value class for case figures, extracted from the news entries."""
    pass

def parse_header(header: str) -> NewsMeta:
    """
    Analyse <header> of the news and return date and time of news or None. 
    
    The <header> is expected to have the following format:
    
    'Aktuelle Lage Stadt und StädteRegion Aachen zum Corona-Virus; Montag, 22. Juni, 10:00 Uhr'
    
    Variables weekday name, date, and time will be extracted from header. Year assumed to be 2020.
    
    Parameters
    ----------
    header : str 
        Extracted raw text of header
    
    Returns
    -------
    None
        If no matching header found
    NewsMeta(weekday_name, day_of_month, month_name, hour, minutes, date, time, datetime)
        Object with meta data of news entry
    """
    
    monthnames = [ "Januar", "Februar", "März", "April", "Mai", "Juni", "Juli", "August", "September", 
                  "Oktober", "November", "Dezember" ]
    
    log.debug("parse_header(" + header[:40] + "...)")
    
    pattern = (r"^Aktuelle Lage Stadt und StädteRegion Aachen zum Corona-Virus;\s*"
               r"(Montag|Dienstag|Mittwoch|Donnerstag|Freitag|Samstag|Sonntag),\s*"
               r"([0-9]{1,2})\.\s*(Januar|Februar|März|April|Mai|Juni|Juli|August|"
               r"September|Oktober|November|Dezember),\s*"
               r"([0-9]{1,2}):([0-9]{2})\s*Uhr.*$")
        
    match = re.search(pattern, header)
    if match:
        meta = NewsMeta()
        try:
            meta.weekday_name = match.group(1)
            meta.day_of_month = int(match.group(2))
            meta.month_name = match.group(3)
            meta.hour = int(match.group(4))
            meta.minutes = int(match.group(5))
            meta.date = date(2020, monthnames.index(meta.month_name) + 1, meta.day_of_month)
            meta.time = time(meta.hour, meta.minutes)
            meta.datetime = datetime.combine(meta.date, meta.time)
            log.debug(f"parse_header/meta.weekday: %s", meta.weekday_name)        
            log.debug(f"parse_header/meta.day_of_month: %d", meta.day_of_month)        
            log.debug(f"parse_header/meta.month_name: %s", meta.month_name)        
            log.debug(f"parse_header/meta.hour: %d", meta.hour)        
            log.debug(f"parse_header/meta.minutes: %d", meta.minutes)        
            log.debug(f"parse_header/meta.date: %s", str(meta.date))        
            log.debug(f"parse_header/meta.time: %s", str(meta.time))
            log.debug(f"parse_header/meta.datetime: %s", str(meta.datetime))
            return meta
        except:
            log.warning("parse_header() failed parsing news header [1]")
            return None
    else: 
        log.warning("parse_header() failed parsing news header [2]")
        return None

def parse_cases(kind: str, meta: NewsMeta, text: str, pattern_total: str, patterns_total_AC: [], 
                pattern_recovered: str, patterns_deaths: [], pattern_active: str) -> CaseFigures:
    """
    Analyse <abstract> or <main> text of the news and return Covid-19 case numbers included therein. 
  
    Parameters
    ----------
    kind : str
        "Main" - Parse main part of news entry
        "Abstract" - Parse abstract of news entry
    meta : NewsMeta
        Meta data of news entry
    text : str
        Text to be parsed
    pattern_total : str
        Regular expression for parsing total number of cases in Städteregion Aachen
    patterns_total_AC : str
        Regular expression for parsing total number of cases in Aachen
    pattern_recovered : str
        Regular expression for parsing total number of recovered cases in Städteregion Aachen
    patterns_deaths : [ str ] 
        Array with regular expressions for parsing total number of deaths in Städteregion Aachen
    pattern_active : str
        Regular expression for parsing number of active cases in Städteregion Aachen
    
    Returns
    -------
    None
        If text could not be parsed successfully
    CaseFigures(total, total_AC, recovered, deaths, active)
        Object with extracted case figures.
    """
    
    log.debug(f"parse_cases(%s, %s)", kind, text[:80])

    if (not text.strip()):
        # Der übergebene Text ist leer
        return None
    
    def log_debug(kind: str, meta: NewsMeta, par_name: str, par_value: int):
        log.debug(f"parse_cases(%s,%s)/figures.%s: %d", kind, str(meta.datetime), par_name, par_value)        

    def log_warn(kind: str, meta: NewsMeta, reason: int):
        log.warning(f"parse_cases(%s,%s) failed [%d]", kind, str(meta.datetime), reason)
        
    figures = CaseFigures()
    
    # Summe der Corona-Fälle in der Städteregion extrahieren
    match = re.search(pattern_total, text)
    if match:
        try:
            figures.total = int(match.group(1))
            log_debug(kind, meta, "total", figures.total)        
        except:
            log_warn(kind, meta, 1)
            return None
    else: 
        log_warn(kind, meta, 2)
        return None
    
    # Summe der Corona-Fälle in der Stadt Aachen extrahieren
    figures.total_AC = None
    for ptac in patterns_total_AC:
        match = re.search(ptac, text)
        if match:
            try:
                figures.total_AC = int(match.group(1))
                log_debug(kind, meta, "total_AC", figures.total_AC)
                break
            except:
                log_warn(kind, meta, 3)
                return None
        else:
            log.debug(f"parse_cases(%s)/total_AC: No match for pattern %s", kind, ptac)
    
    if figures.total_AC is None:
        log_warn(kind, meta, 4)
    
    # Summe der wieder Genesenen extrahieren
    match = re.search(pattern_recovered, text)
    if match:
        try:
            figures.recovered = int(match.group(1))
            log_debug(kind, meta, "recovered", figures.recovered)        
        except:
            log_warn(kind, meta, 5)
            return None
    else: 
        log_warn(kind, meta, 6)
        return None
    
    # Summe der Corona-Toten extrahieren
    figures.deaths = None
    for pd in patterns_deaths:
        match = re.search(pd, text)
        if match:
            try:
                figures.deaths = int(match.group(1))
                log_debug(kind, meta, "deaths", figures.deaths)
                break
            except:
                log_warn(kind, meta, 7)
                return None
        else:
            log.debug(f"parse_cases(%s)/deaths: No match for pattern %s", kind, pd)

    if figures.deaths is None:
        log_warn(kind, meta, 8)
        return None
    
    # Anzahl der aktiven Infektionen extrahieren
    match = re.search(pattern_active, text)
    if match:
        try:
            figures.active = int(match.group(1))
            log_debug(kind, meta, "active", figures.active)        
        except:
            log_warn(kind, meta, 9)
            return None
    else:
        log_warn(kind, meta, 10)
        return None

    return figures

def parse_abstract(abstract: str, meta: NewsMeta) -> CaseFigures:
    """
    Analyse <abstract> of the news and return Covid-19 case numbers included in abstract. 

    The relevant area of the <abstract> is expected to have the following format:
    
    'Aktuell 1999 bestätigte Coronafälle in der StädteRegion Aachen (davon 994 in der Stadt Aachen). '
    '1880 ehemals positiv auf das Corona-Virus getestete Personen sind inzwischen wieder gesund.'
    'Bislang 98 Todesfälle.' | 'neuer Todesfall, somit insgesamt 98.' | 'neue Todesfälle, somit insgesamt 98.' 
    'Damit aktuell 21 nachgewiesen(e) Infizierte.'
    
    Values will be extracted from abstract text. 
    
    Parameters
    ----------
    meta : NewsMeta
        Meta data of news entry
    abstract : str
        Abstract text to be parsed
    
    Returns
    -------
    None
        If text could not be parsed successfully
    CaseFigures(total, total_AC, recovered, deaths, active)
        Object with extracted case figures.
    """
    
    log.debug("parse_abstract(" + abstract[:40] + "...)")
    
    pattern_total = r"Aktuell\s*([0-9]{1,6})\s*bestätigte\s*Coronafälle\s*in\s*der\s*StädteRegion\s*"
    patterns_total_AC = [
        r"davon\s*([0-9]{1,6})\s*in\s*der\s*Stadt\s*Aachen"
    ]
    pattern_recovered = (r"([0-9]{1,6})\s*ehemals\s*positiv\s*auf\s*das\s*Corona-Virus\s*getestete\s*"
                         r"Personen\s*sind\s*inzwischen\s*wieder\s*gesund.*")
    pattern_deaths = [
        r"(?:Insgesamt|Bislang)\s*([0-9]{1,6})\s*Todesfälle\.",
        r"(?:neuer\s*Todesfall|neue\s*Todesfälle),\s*somit\s*insgesamt\s*([0-9]{1,6})\."
    ]
    pattern_active = r"Damit\s*aktuell\s*([0-9]{1,6})\s*nachgewiesene?\s*Infizie.*"
        
    return parse_cases("Abstract", meta, abstract, 
                       pattern_total, patterns_total_AC, 
                       pattern_recovered, pattern_deaths, pattern_active)

def parse_main(main: str, meta: NewsMeta) -> CaseFigures:
    """
    Analyse <main> text of the news and return Covid-19 case numbers included in main text. 
    
    The relevant area of the <abstract> is expected to have the following format:
      
    'Es gibt insgesamt in der StädteRegion [nunmehr] 1997 positive Fälle, davon 992 in der Stadt Aachen'
    '1876 ehemals positiv auf das Corona-Virus getestete Personen sind inzwischen wieder gesund'
    'Die Zahl der gemeldeten Todesfälle liegt [nach wie vor] bei 98'
    'Damit sind aktuell 23 Menschen in der StädteRegion nachgewiesen infiziert'

    Parameters
    ----------
    meta : NewsMeta
        Meta data of news entry
    main : str
        Main text to be parsed
    
    Returns
    -------
    None
        If text could not be parsed successfully
    CaseFigures(total, total_AC, recovered, deaths, active)
        Object with extracted case figures.
    """
    log.debug("parse_main(" + main[:40] + "...)")

    pattern_total = (r"Es\s*gibt\s*insgesamt\s*in\s*der\s*StädteRegion\s*(?:nunmehr)?\s*([0-9]{1,6})\s*"
                     r"positive\s*Fälle")
    patterns_total_AC = [
        r"davon\s*([0-9]{1,6})\s*in\s*der\s*Stadt\s*Aachen",
        r"Aachen\s*(?:[0-9]{1,6})\s*\(([0-9]{1,6})\)"
    ]
    pattern_recovered = (r"([0-9]{1,6})\s*ehemals\s*positiv\s*auf\s*das\s*Corona-Virus\s*getestete\s*"
                         r"Personen\s*sind\s*inzwischen\s*wieder\s*gesund")
    patterns_deaths = [
        r"Die\s*Zahl\s*der\s*gemeldeten\s*Todesfälle\s*liegt\s*(?:nach\s*wie\s*vor)?\s*bei\s*([0-9]{1,6})"
    ]
    pattern_active = (r"Damit\s*sind\s*aktuell\s*([0-9]{1,6})\s*Menschen\s*in\s*der\s*StädteRegion\s*"
                      r"(?:Aachen)?\s*nachgewiesen\s*infiziert")
    
    return parse_cases("Main", meta, main, 
                       pattern_total, patterns_total_AC, 
                       pattern_recovered, patterns_deaths, pattern_active)

def parse_body(body: str, meta: NewsMeta) -> CaseFigures:
    """
    Analyse <body> text of the news and return Covid-19 case numbers included in main text. 
    
    The relevant area of the <body> is expected to have the following format:
      
    'Es gibt insgesamt in der StädteRegion [nunmehr] 1997 positive Fälle, davon 992 in der Stadt Aachen'
    '1876 ehemals positiv auf das Corona-Virus getestete Personen sind inzwischen wieder gesund'
    'Die Zahl der gemeldeten Todesfälle liegt [nach wie vor] bei 98'
    'Damit sind aktuell 23 Menschen in der StädteRegion nachgewiesen infiziert'

    Parameters
    ----------
    meta : NewsMeta
        Meta data of news entry
    Body : str
        Text in news body to be parsed
    
    Returns
    -------
    None
        If text could not be parsed successfully
    CaseFigures(total, total_AC, recovered, deaths, active)
        Object with extracted case figures
    """
    log.debug("parse_body(" + body[:40] + "...)")

    pattern_total = (r"Es\s*gibt\s*insgesamt\s*in\s*der\s*StädteRegion\s*(?:nunmehr)?\s*([0-9]{1,6})\s*"
                     r"positive\s*Fälle")
    patterns_total_AC = [
        r"davon\s*([0-9]{1,6})\s*in\s*der\s*Stadt\s*Aachen",
        r"Aachen\s*(?:[0-9]{1,6})\s*\(([0-9]{1,6})\)"
    ]
    pattern_recovered = (r"([0-9]{1,6})\s*ehemals\s*positiv\s*auf\s*das\s*Corona-Virus\s*getestete\s*"
                         r"Personen\s*sind\s*inzwischen\s*wieder\s*gesund")
    patterns_deaths = [
        r"Die\s*Zahl\s*der\s*gemeldeten\s*Todesfälle\s*liegt\s*(?:nach\s*wie\s*vor)?\s*bei\s*([0-9]{1,6})"
    ]
    pattern_active = (r"Damit\s*sind\s*aktuell\s*([0-9]{1,6})\s*Menschen\s*in\s*der\s*StädteRegion\s*"
                      r"(?:Aachen)?\s*nachgewiesen\s*infiziert")
    
    return parse_cases("Body", meta, body, 
                       pattern_total, patterns_total_AC, 
                       pattern_recovered, patterns_deaths, pattern_active)

def checked_data1(meta: NewsMeta, fig_abstract: CaseFigures, fig_main: CaseFigures) -> []:
    """
    Check extracted data for consistency and return valid figures or None.
    
    Parameters
    ----------
    meta : NewMeta
        Meta data of news entry
    fig_abstract: CaseFigures
        Object with figures extracted from abstract of the news entry
    fig_main: CaseFigures
        Object with figures extracted from main text of the news entry
    
    Returns
    -------
    None 
        If consistency check fails
    [ Date, Time, Total, Total_AC, Deaths, Recovered, Active ] 
        Array with extracted Covid-19 numbers
    """
    
    if meta != None: 
        # Der Header konnte erfolgreich analysiert werden
        a, m = fig_abstract, fig_main
        if (a != None) and (m != None): 
            # Sowohl Abstract als auch Main konnten erfolgreich analysiert werden
            if (m.total == a.total) and (m.deaths == a.deaths) and (m.recovered == a.recovered) and \
               (m.active == a.active):
                # Die eingelesenen Zahlen aus Abstract und Main stimmmen überein
                if m.active == 0:
                    # Die Anzahl aktiver Fälle war nicht explizit angegeben und wird stattdessen berechnet
                    m.active = m.total - m.deaths - m.recovered
                if m.total - m.deaths - m.recovered == m.active:
                    # Die eingelesenen Daten haben die Konsistenzprüfung bestanden
                    return [ meta.date, meta.time, m.total, m.total_AC, m.deaths, m.recovered, m.active ]
                else:
                    log.warning('Data consistency error: (1) Main and Abstract found. Inconsistent figures read.')
                    return None
        elif m != None: 
            # Mindestens Main konnte erfolgreich analysiert werden
            if m.active == 0:
                # Die Anzahl aktiver Fälle war nicht explizit angegeben und wird stattdessen berechnet
                m.active = m.total - m.deaths - m.recovered
            if m.total - m.deaths - m.recovered == m.active:
                # Die eingelesenen Daten haben die Konsistenzprüfung bestanden
                return [ meta.date, meta.time, m.total, m.total_AC, m.deaths, m.recovered, m.active ]
            else:
                log.warning('Data onsistency error: (2) Only Main found. Inconsistent figures read. ')
                return None
    else:
        log.warning('Data consistency error: (3) Header not found.')
        return None

def checked_data2(meta: NewsMeta, fig_body: CaseFigures) -> []:
    """
    Check extracted data for consistency and return valid figures or None.
    
    Parameters
    ----------
    meta : NewMeta
        Meta data of news entry
    fig_body: CaseFigures
        Object with figures extracted from body of the news entry
    
    Returns
    -------
    None 
        If consistency check fails
    [ Date, Time, Total, Total_AC, Deaths, Recovered, Active ] 
        Array with extracted Covid-19 numbers
    """
    
    if meta != None: 
        # Der Header konnte erfolgreich analysiert werden
        if fig_body != None:
            if fig_body.active == 0:
                # Die Anzahl aktiver Fälle war nicht explizit angegeben und wird stattdessen berechnet
                fig_body.active = fig_body.total - fig_body.deaths - fig_body.recovered
            if fig_body.total - fig_body.deaths - fig_body.recovered == fig_body.active:
                # Die eingelesenen Daten haben die Konsistenzprüfung bestanden
                return [
                    meta.date, meta.time, 
                    fig_body.total, fig_body.total_AC, fig_body.deaths, fig_body.recovered, fig_body.active
                ]
            else:
                log.warning('Data onsistency error: Inconsistent figures read. ')
                return None
    else:
        log.warning('Data consistency error: Header not found.')
        return None

def fill_dataframe_from_text(text_records: []) -> pd.DataFrame:
    """
    Analyse <text_records> and return Pandas dataframe with parsed Covid-19 case numbers. 
    
    Parameters
    ----------
    text_records : []
        Array with extracted text
        
    Returns
    -------
    pd.DataFrame
        Pandas dataframe with extracted data
    """
    
    log.debug(f"fill_dataframe_from_text(%d records)", len(text_records))

    figures = [] # Zwischenspeicherung der eingelesenen Zahlen
    dates = [] # Zwischenspeicherung der Datumsangaben
    
    for tr in text_records:
        meta = parse_header(tr['Header'])
        
        if tr['Body']:
            fig_body = parse_body(tr['Body'], meta)
            rec = checked_data2(meta, fig_body)
        else:    
            if tr['Abstract']:
                fig_abstract = parse_abstract(tr['Abstract'], meta)
            if tr['Main']:
                fig_main = parse_main(tr['Main'], meta)
            rec = checked_data1(meta, fig_abstract, fig_main)
            
        if rec:
            dates.append(rec[0])
            figures.append(rec[1:])

    if figures.count:
        cols = ['Uhrzeit', 'Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ]
        index = pd.DatetimeIndex(dates, name='Datum')
        df = pd.DataFrame(np.array(figures), columns=cols, index=index).sort_index()
        return df
    
    return pd.DataFrame() # Leer

## Datenabfrage durchführen

1. Datum des letzten Datensatzes in der Excel-Datei ermitteln und mit aktuellem Datum vergleichen. 
2. Wenn mindestens 1 Tag seit der letzten Datenabfrage vergangen ist, neue Abfrage durchführen.
3. Von der Website gelieferte Rohtexte auswerten und Fallzahlen extrahieren
4. Extrahierte Fallzahlen auf Konsistenz prüfen
5. Geprüfte Fallzahlen zu neuem DataFrame hinzufügen
6. Den neuen mit dem existierendem DataFrame zusammenführen
7. Zusammengeführte Daten speichern

In [31]:
new_request = (c19_cases.size == 0) or \
    (date.today() >= (date.fromtimestamp(c19_cases.index.max().timestamp()) + timedelta(days=1)))

if new_request:
    log.info("New request initiated")
    if not html_payload:
        url = config['Rohdaten']['SourceURLDefault']
        if robot_access_allowed(url):
            html_payload = gather_html(url)

    if html_payload:
        records = gather_text(html_payload)
        
    log.info("Processing {0} text records".format(len(records)))
    if len(records):
        new_cases = fill_dataframe_from_text(records)

    if not new_cases.empty:
        # Nur Zeilen mit neuerem Datum hinzufügen
        merged_cases = pd.concat([c19_cases, new_cases[new_cases.index > c19_cases.index[-1]]], join='outer')
        log.info("{0} new case records appended".format(len(merged_cases.index) - len(c19_cases.index)))
    else:
        log.info("No new case figures extracted")
    
    merged_cases.to_excel(config['Rohdaten']['FileName'], 
                          sheet_name=config['Rohdaten']['SheetName'], index_label='Datum')
    
    merged_cases.tail(10)
else:
    log.info("No new request required")

Aktuelle Zahlen:
Aktuelle Zahlen:Sieben-Tage-Inzidenz:
Aktuelle Zahlen:
Aktuelle Zahlen:Die Fälle verteilen sich wie folgt auf die Kommunen (Kommune: Aktive Fälle/Gesamtzahl der Fälle): Aachen: 35/1106, Alsdorf: 3/123, Baesweiler: 9/102, Eschweiler: 13/175, Herzogenrath: 5/210, Monschau: 1/64, Roetgen: 0/21, Simmerath: 1/79, Stolberg: 12/183, Würselen: 6/198
Aktuelle Zahlen:Die Fälle verteilen sich wie folgt auf die Kommunen (Kommune: Aktive Fälle/Gesamtzahl der Fälle): Aachen: 35/1106, Alsdorf: 3/123, Baesweiler: 9/102, Eschweiler: 13/175, Herzogenrath: 5/210, Monschau: 1/64, Roetgen: 0/21, Simmerath: 1/79, Stolberg: 12/183, Würselen: 6/198Sieben-Tage-Inzidenz:
Aktuelle Zahlen:Die Fälle verteilen sich wie folgt auf die Kommunen (Kommune: Aktive Fälle/Gesamtzahl der Fälle): Aachen: 35/1106, Alsdorf: 3/123, Baesweiler: 9/102, Eschweiler: 13/175, Herzogenrath: 5/210, Monschau: 1/64, Roetgen: 0/21, Simmerath: 1/79, Stolberg: 12/183, Würselen: 6/198Sieben-Tage-Inzidenz:Coronaschutzverordnu

ValueError: Empty data passed with indices specified.